In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
date = "2023-11"

my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key!

In [3]:
end_use = "naics?get=CON_VAL_MO,CTY_CODE,CTY_NAME,SUMMARY_LVL"

url = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 
url = url + my_key + "&time==from+2013-01"

r = requests.get(url) 
    
print(r)
    
df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

df.columns = r.json()[0]

df["total_imports"] = df["CON_VAL_MO"].astype(float)

df = df[df.SUMMARY_LVL == "DET"]

grp = df.groupby(["CTY_NAME"])

top_products = grp.agg({"total_imports":"sum","CTY_CODE":"first"})

country_list = list(top_products.sort_values(by = "total_imports", ascending = False).CTY_CODE)[0:31]


['TOTAL FOR ALL COUNTRIES','NAFTA','EUROPEAN UNION']

<Response [200]>


['TOTAL FOR ALL COUNTRIES', 'NAFTA', 'EUROPEAN UNION']

In [4]:
country_list[0] = ""

In [5]:
country_list.extend(["0003", "0020"])

In [6]:
country_list

['',
 '5700',
 '2010',
 '1220',
 '5880',
 '4280',
 '5800',
 '5520',
 '5330',
 '4120',
 '5830',
 '4190',
 '4759',
 '4279',
 '4419',
 '5570',
 '5490',
 '3510',
 '5590',
 '4210',
 '5600',
 '5170',
 '5081',
 '4231',
 '4621',
 '4700',
 '3010',
 '4330',
 '4010',
 '5650',
 '6021',
 '0003',
 '0020']

In [7]:
end_use = "hs?get=CTY_NAME,CON_VAL_MO,CAL_DUT_MO,I_COMMODITY,I_COMMODITY_SDESC"

surl = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 

surl  = surl + my_key + "&time=" + "from+2013-01" + "&COMM_LVL=HS2" 

for xxx in country_list:
    
    out_file = ".\\data"+ "\\imports\\" + xxx + "data-" + date + ".parquet"
    
    if xxx == "":
        out_file = ".\\data"+ "\\imports\\" + "TOTAL" + "data-" + date + ".parquet"
    
    
    if os.path.exists(out_file):
        
        print("Already have downloaded file")
        
        continue
    
    print(xxx)
    
    url = surl + "&CTY_CODE=" + xxx
    
    if xxx == "":
        url = surl
    
    r = requests.get(url) 
    
    print(r)
    
    foo = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    foo.columns = r.json()[0]

    pq.write_table(pa.Table.from_pandas(foo), out_file)

Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
Already have downloaded file
0020
<Response [200]>


C:\ProgramData\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:383: FutureWarning: RangeIndex._start is deprecated and will be removed in a future version. Use RangeIndex.start instead
  'start': level._start,
C:\ProgramData\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:384: FutureWarning: RangeIndex._stop is deprecated and will be removed in a future version. Use RangeIndex.stop instead
  'stop': level._stop,
C:\ProgramData\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:385: FutureWarning: RangeIndex._step is deprecated and will be removed in a future version. Use RangeIndex.step instead
  'step': level._step


In [8]:
foo

,CTY_NAME,CON_VAL_MO,CAL_DUT_MO,I_COMMODITY,I_COMMODITY_SDESC,time,COMM_LVL,CTY_CODE
0,NAFTA,182150721,14,01,LIVE ANIMALS,2013-01,HS2,0020
1,NAFTA,212532311,156,01,LIVE ANIMALS,2013-02,HS2,0020
2,NAFTA,244602053,57,01,LIVE ANIMALS,2013-03,HS2,0020
3,NAFTA,250009423,199,01,LIVE ANIMALS,2013-04,HS2,0020
4,NAFTA,182211881,50,01,LIVE ANIMALS,2013-05,HS2,0020
...,...,...,...,...,...,...,...,...
12831,NAFTA,170499273,524416,95,"TOYS, GAMES & SPORT EQUIPMENT; PARTS & ACCESSO...",2023-07,HS2,0020
12832,NAFTA,187315621,631024,95,"TOYS, GAMES & SPORT EQUIPMENT; PARTS & ACCESSO...",2023-08,HS2,0020
12833,NAFTA,170871541,326635,95,"TOYS, GAMES & SPORT EQUIPMENT; PARTS & ACCESSO...",2023-09,HS2,0020
12834,NAFTA,176350969,493105,95,"TOYS, GAMES & SPORT EQUIPMENT; PARTS & ACCESSO...",2023-10,HS2,0020


In [9]:
country_list[0] = "TOTAL"

df = pd.DataFrame([])
#this is for all countries...

for xxx in country_list:
    
    file = ".\\data\\imports\\" + xxx + "data-" + date + ".parquet"

    foo = pq.read_table(file).to_pandas()
    
    #recentfile = ".\\data\\imports\\" + xxx + "-" + date + "-" + "data.parquet"
    
    #foo = foo.append(pq.read_table(recentfile).to_pandas())
    
    #pq.write_table(pa.Table.from_pandas(foo), file)
    #this combines the two periods
    
    df = df.append(foo)
            
df.time = pd.to_datetime(df.time, format="%Y-%m")

df["imports"] = df["CON_VAL_MO"].astype(float)

df["duty"] = df["CAL_DUT_MO"].astype(float)

total = df.groupby(["CTY_NAME", "time"]).agg({"imports": "sum", "duty": "sum"})
                   
total.reset_index(inplace = True)
                   
total["I_COMMODITY"] = "00"
total["I_COMMODITY_SDESC"] = "ALL PRODUCTS"
                   
df = df.append(total)

df["itariff"] = 100*df["duty"]/df["imports"]

df.drop(["CON_VAL_MO", "COMM_LVL", "CTY_CODE", "CAL_DUT_MO"], axis = 1, inplace = True)

df["I_COMMODITY_SDESC"] = "HS CODE " + df["I_COMMODITY"] + ", " + df["I_COMMODITY_SDESC"]

df["I_COMMODITY_SDESC"].replace({"HS CODE 00, ALL PRODUCTS": "ALL PRODUCTS"}, inplace = True)

df.set_index(["CTY_NAME","I_COMMODITY_SDESC","time"],inplace = True)

out_file = ".\\data"+ "\\top30-HS2-imports.parquet"

pq.write_table(pa.Table.from_pandas(df), out_file)

In [11]:
df.loc["NAFTA","ALL PRODUCTS"]["2023"]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  """Entry point for launching an IPython kernel.


,I_COMMODITY,imports,duty,itariff
time,,,,
2023-01-01,00,7.116719e+10,125062268.0,0.175730
2023-02-01,00,6.782299e+10,116521116.0,0.171802
2023-03-01,00,8.048363e+10,136139047.0,0.169151
2023-04-01,00,7.252259e+10,120110278.0,0.165618
2023-05-01,00,7.793375e+10,136911038.0,0.175676
2023-06-01,00,7.618537e+10,136652297.0,0.179368
2023-07-01,00,7.177608e+10,143234834.0,0.199558
2023-08-01,00,7.761343e+10,148751548.0,0.191657
2023-09-01,00,7.432701e+10,136239418.0,0.183297


In [12]:
file = ".\\data\\imports\\" + "TOTAL" + "data.parquet"

foo = pq.read_table(file).to_pandas()
    
foo

,CTY_NAME,CON_VAL_MO,CAL_DUT_MO,I_COMMODITY,I_COMMODITY_SDESC,time,COMM_LVL
0,TOTAL FOR ALL COUNTRIES,784536648,4504327,07,EDIBLE VEGETABLES & CERTAIN ROOTS & TUBERS,2013-01,HS2
1,TOTAL FOR ALL COUNTRIES,315892605,2650541,17,SUGARS AND SUGAR CONFECTIONARY,2013-01,HS2
2,TOTAL FOR ALL COUNTRIES,334605750,723046,10,CEREALS,2013-01,HS2
3,TOTAL FOR ALL COUNTRIES,144113171,409478,13,"LAC; GUMS, RESINS & OTHER VEGETABLE SAP & EXTRACT",2013-01,HS2
4,TOTAL FOR ALL COUNTRIES,371169570,2283401,18,COCOA AND COCOA PREPARATIONS,2013-01,HS2
...,...,...,...,...,...,...,...
9991,TOTAL FOR ALL COUNTRIES,794906854,2374814,09,"COFFEE, TEA, MATE & SPICES",2021-06,HS2
9992,TOTAL FOR ALL COUNTRIES,146629956,1172022,10,CEREALS,2021-06,HS2
9993,TOTAL FOR ALL COUNTRIES,186372088,2220530,11,MILLING PRODUCTS; MALT; STARCH; INULIN; WHT GL...,2021-06,HS2
9994,TOTAL FOR ALL COUNTRIES,340184476,3851904,12,"OIL SEEDS ETC.; MISC GRAIN, SEED, FRUIT, PLANT...",2021-06,HS2


In [13]:
foo = pq.read_table(".\\data\\top30-HS2-imports.parquet").to_pandas()

In [14]:
foo.tail()

I_COMMODITY       imports         duty  \
CTY_NAME I_COMMODITY_SDESC time                                                
VIETNAM  ALL PRODUCTS      2023-07-01          00  9.963163e+09  429336424.0   
                           2023-08-01          00  9.955884e+09  458647276.0   
                           2023-09-01          00  1.052066e+10  425070758.0   
                           2023-10-01          00  1.077170e+10  383603444.0   
                           2023-11-01          00  1.036588e+10  360602430.0   

                                        itariff  
CTY_NAME I_COMMODITY_SDESC time                  
VIETNAM  ALL PRODUCTS      2023-07-01  4.309238  
                           2023-08-01  4.606796  
                           2023-09-01  4.040342  
                           2023-10-01  3.561214  
                           2023-11-01  3.478745

In [20]:
foo.loc["TOTAL FOR ALL COUNTRIES", "HS CODE 01, LIVE ANIMALS"]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  """Entry point for launching an IPython kernel.


,I_COMMODITY,imports,duty,itariff
time,,,,
2013-01-01,01,221313094.0,417.0,0.000188
2013-02-01,01,246092109.0,823.0,0.000334
2013-03-01,01,261244392.0,75.0,0.000029
2013-04-01,01,271133926.0,1389.0,0.000512
2013-05-01,01,209785289.0,50.0,0.000024
...,...,...,...,...
2023-07-01,01,262122850.0,76233.0,0.029083
2023-08-01,01,315165591.0,57092.0,0.018115
2023-09-01,01,363074240.0,94337.0,0.025983
